In [1]:
import pandas as pd

root = '../dataset/hi'

# Load dataset
hiv = pd.read_csv(f'{root}/hiv-charge-CS2_36.csv')
hii = pd.read_csv(f'{root}/hii-charge-CS2_36.csv')

In [2]:
# Unisci i due dataset utilizzando la colonna "cycle"
df = pd.merge(hiv, hii, on="cycle")

# Rimuovi la colonna "cycle" dal DataFrame risultante
df = df.drop(columns=["cycle"], inplace=False)

# Visualizza il DataFrame risultante
print(df)

         hi_v      hi_i
0    7.091993  0.125641
1    6.639214  0.121732
2    6.599330  0.120788
3    6.627282  0.120991
4    6.389022  0.121652
..        ...       ...
955  0.371274  0.109306
956  0.345076  0.109547
957  0.392511  0.110228
958  0.404034  0.110994
959  0.399089  0.110416

[960 rows x 2 columns]


In [3]:
# Split into train and test sets
train_size = int(len(df) * 0.8)
train_data, test_data = df[:train_size], df[train_size:]

In [4]:
from sklearn.preprocessing import StandardScaler

# Istanzia lo StandardScaler
scaler = StandardScaler()

# Standardizza i dati
scaled_data = scaler.fit_transform(df)

# Crea un nuovo DataFrame con i dati standardizzati
df_standardized = pd.DataFrame(scaled_data, columns=df.columns)


c:\Python39\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Python39\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Python39\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Python39\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.appl

In [5]:
df

,hi_v,hi_i
0,7.091993,0.125641
1,6.639214,0.121732
2,6.599330,0.120788
3,6.627282,0.120991
4,6.389022,0.121652
...,...,...
955,0.371274,0.109306
956,0.345076,0.109547
957,0.392511,0.110228
958,0.404034,0.110994


In [16]:
import numpy as np

def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length +1):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

In [18]:
seq_length = 10

# Create sequences for training set
X_train, y_train = create_sequences(train_data, seq_length)

# Create sequences for testing set
X_test, y_test = create_sequences(test_data, seq_length)

# Reshape input data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

KeyError: 10